In [1]:
using Gridap
using GridapGmsh
using Gridap.Io
using LinearAlgebra

In [2]:
function Gridap.DiscreteModelFromFile(filename::AbstractString,::Val{:msh})
  model = GmshDiscreteModel(filename)
  model
end

In [3]:
#the function for the dirichlet boundary conditions as is defined in the excersice sheet
g(x)=sum(x)

g (generic function with 1 method)

In [4]:
 # here it goes the function of the right side of the poisson eq
#this is only a test function, we can chosse whichever and solve the problem
#because here we are trying to measure the run time
f(x) = norm(x)

f (generic function with 1 method)

In [5]:

function solve_poisson(mesh_file; order=1)
  # Load the mesh
  @info "Loading mesh file $mesh_file"
  model = GmshDiscreteModel(mesh_file)
  

  reffe = ReferenceFE(lagrangian, Float64, order)
  V0 = TestFESpace(model, reffe;conformity=:H1, dirichlet_tags=["wall"])
  U = TrialFESpace(V0, g)
    degree = 2 * order 
  Ω = Triangulation(model)
  dΩ = Measure(Ω, degree)

  
  a(u,v) = ∫( ∇(v) ⋅ ∇(u) ) * dΩ
  b(v) = ∫( v*f ) * dΩ
  op = AffineFEOperator(a, b, U, V0)

 
  uh = solve(op)


  # write the output as the entrance to identify each one
  filename = first(splitext(mesh_file)) * "_solution.vtu"
  writevtk(Ω, filename, cellfields=[ "uh" => uh])

  return op
end

solve_poisson (generic function with 1 method)

In [186]:
rec=solve_poisson("rectangulo.msh")
Ar=get_matrix(rec)
br=get_vector(rec)
Ar_dense=Matrix(Ar);
tsr = @elapsed begin cholesky(Ar)\br end

tmr = @elapsed begin lu(Ar_dense)\br end
abs(tsr-tmr)

┌ Info: Loading mesh file rectangulo.msh
└ @ Main In[185]:4


Info    : Reading 'rectangulo.msh'...
Info    : 9 entities
Info    : 729 nodes
Info    : 1456 elements
Info    : Done reading 'rectangulo.msh'


0.00416795

In [7]:
cub=solve_poisson("cub05.msh")
Ac=get_matrix(cub)
bc=get_vector(cub)
Ac_dense=Matrix(Ac);
tsc = @elapsed begin cholesky(Ac)\bc end

tmc = @elapsed begin lu(Ac_dense)\bc end
abs(tsc-tmc)

┌ Info: Loading mesh file cub05.msh
└ @ Main In[5]:4


Info    : Reading 'cub05.msh'...
Info    : 27 entities
Info    : 1843 nodes
Info    : 8089 elements
Info    : Done reading 'cub05.msh'


0.10156817500000004

In [188]:
lin=solve_poisson("line.msh")
Al=get_matrix(lin)
bl=get_vector(lin)
Al_dense=Matrix(Al);
tsl = @elapsed begin cholesky(Al)\bl end

tml = @elapsed begin lu(Al_dense)\bl end

┌ Info: Loading mesh file line.msh
└ @ Main In[185]:4


Info    : Reading 'line.msh'...
Info    : 3 entities
Info    : 11 nodes
Info    : 12 elements
Info    : Done reading 'line.msh'


1.6621e-5

In [ ]:
file_name=["cube1","cube2","cube3","cube4"]
for 